# Testing Orbit Controller SDK
    

### Author: AWS Professional Services Emerging Technology and Intelligent Platforms Group
### Date: Mar 10 2021

In [1]:
import os
import sys
import boto3
from aws_orbit_sdk import controller
from aws_orbit_sdk.common import get_workspace,get_scratch_database
from aws_orbit_sdk.magics.orbit import OrbitWorkbenchMagics 

import json
import time

env_name = %env AWS_ORBIT_ENV
team_name = %env AWS_ORBIT_TEAM_SPACE
(env_name,team_name)

('dev-env', 'lake-user')

In [2]:
%%run_notebook
{    
      "tasks":  [
            {
                  "notebookName": "Example-1-SQL-Analysis-Athena.ipynb",
                  "sourcePath": "shared/samples/notebooks/B-DataAnalyst",
                  "targetPath": "shared/regression/notebooks/B-DataAnalyst",
                  "targetPrefix": "ttt",
                  "params": {
                        "glue_db" : "cms_raw_db",
                        "target_db" : "users"
                  }      
            }
      ]  
}

INFO:root:using default profile {'display_name': 'Micro', 'slug': 'micro', 'description': '2 CPU + 2G MEM', 'kubespawner_override': {'cpu_guarantee': 2, 'cpu_limit': 2, 'mem_guarantee': '2G', 'mem_limit': '2G'}, 'default': True}
INFO:root:volumes:[{"name": "efs-volume", "persistentVolumeClaim": {"claimName": "jupyterhub"}}]
INFO:root:volume_mounts:[{"mountPath": "/efs", "name": "efs-volume"}]


{'ExecutionType': 'eks',
 'Identifier': 'orbit-lake-user-fargate-runner-snd75',
 'NodeType': 'fargate',
 'tasks': [{'notebookName': 'Example-1-SQL-Analysis-Athena.ipynb',
   'sourcePath': 'shared/samples/notebooks/B-DataAnalyst',
   'targetPath': 'shared/regression/notebooks/B-DataAnalyst',
   'targetPrefix': 'ttt',
   'params': {'glue_db': 'cms_raw_db', 'target_db': 'users'}}]}

In [3]:
job_name=Out[2]['Identifier']
job_name

'orbit-lake-user-fargate-runner-snd75'

In [4]:
%%schedule_notebook -cron 0/2 * 1/1 * ?  -id ttt
{    
      "tasks":  [
            {
                  "notebookName": "Example-1-SQL-Analysis-Athena.ipynb",
                  "sourcePath": "shared/samples/notebooks/B-DataAnalyst",
                  "targetPath": "shared/regression/notebooks/B-DataAnalyst",
                  "targetPrefix": "ttt",
                  "params": {
                        "glue_db" : "cms_raw_db",
                        "target_db" : "users"
                  }      
            }
      ]  
}


INFO:root:using default profile {'display_name': 'Micro', 'slug': 'micro', 'description': '2 CPU + 2G MEM', 'kubespawner_override': {'cpu_guarantee': 2, 'cpu_limit': 2, 'mem_guarantee': '2G', 'mem_limit': '2G'}, 'default': True}
INFO:root:volumes:[{"name": "efs-volume", "persistentVolumeClaim": {"claimName": "jupyterhub"}}]
INFO:root:volume_mounts:[{"mountPath": "/efs", "name": "efs-volume"}]


{'ExecutionType': 'eks', 'Identifier': 'orbit-lake-user-ttt'}

In [5]:
cronjob_name=Out[4]['Identifier']
cronjob_name

'orbit-lake-user-ttt'

In [6]:
jobs = controller.list_my_running_jobs()
assert(len(jobs) > 0)
jobs

[{'metadata': {'name': 'orbit-lake-user-fargate-runner-snd75',
   'generateName': 'orbit-lake-user-fargate-runner-',
   'namespace': 'lake-user',
   'selfLink': '/apis/batch/v1/namespaces/lake-user/jobs/orbit-lake-user-fargate-runner-snd75',
   'uid': 'b372bbb2-6698-4d75-9cd9-32434d0b1cb9',
   'resourceVersion': '1319169',
   'creationTimestamp': '2021-03-09T21:50:48Z',
   'labels': {'app': 'orbit-runner',
    'orbit/node-type': 'fargate',
    'username': 'amirbo'},
   'managedFields': [{'manager': 'OpenAPI-Generator',
     'operation': 'Update',
     'apiVersion': 'batch/v1',
     'time': '2021-03-09T21:50:48Z',
     'fieldsType': 'FieldsV1',
     'fieldsV1': {'f:metadata': {'f:generateName': {},
       'f:labels': {'.': {},
        'f:app': {},
        'f:orbit/node-type': {},
        'f:username': {}}},
      'f:spec': {'f:backoffLimit': {},
       'f:completions': {},
       'f:parallelism': {},
       'f:template': {'f:metadata': {'f:labels': {'.': {},
          'f:app': {},
     

In [7]:
jobs = controller.list_team_running_jobs()
print(jobs)
assert(len(jobs) == 0)

[]


In [8]:
cronjobs = controller.list_running_cronjobs()
assert(len(cronjobs) > 0)
cronjobs

[{'metadata': {'name': 'orbit-lake-user-ttt',
   'namespace': 'lake-user',
   'selfLink': '/apis/batch/v1beta1/namespaces/lake-user/cronjobs/orbit-lake-user-ttt',
   'uid': '39ad2bb6-2418-44de-b995-85386bca46ce',
   'resourceVersion': '1319172',
   'creationTimestamp': '2021-03-09T21:50:48Z',
   'labels': {'app': 'orbit-runner',
    'cronjob_id': 'orbit-lake-user-ttt',
    'orbit/node-type': 'fargate',
    'username': 'amirbo'},
   'managedFields': [{'manager': 'OpenAPI-Generator',
     'operation': 'Update',
     'apiVersion': 'batch/v1beta1',
     'time': '2021-03-09T21:50:48Z',
     'fieldsType': 'FieldsV1',
     'fieldsV1': {'f:metadata': {'f:labels': {'.': {},
        'f:app': {},
        'f:cronjob_id': {},
        'f:orbit/node-type': {},
        'f:username': {}}},
      'f:spec': {'f:concurrencyPolicy': {},
       'f:failedJobsHistoryLimit': {},
       'f:jobTemplate': {'f:spec': {'f:backoffLimit': {},
         'f:template': {'f:metadata': {'f:labels': {'.': {},
            'f

In [9]:
!kubectl get cronjob 

NAME                  SCHEDULE         SUSPEND   ACTIVE   LAST SCHEDULE   AGE
orbit-lake-user-ttt   0/2 * 1/1 * ?    False     0        <none>          1s


In [10]:
controller.delete_cronjob(job_name=cronjob_name)

In [11]:
!kubectl get cronjob 

No resources found in lake-user namespace.


In [12]:
!kubectl describe job $job_name

Name:           orbit-lake-user-fargate-runner-snd75
Namespace:      lake-user
Selector:       controller-uid=b372bbb2-6698-4d75-9cd9-32434d0b1cb9
Labels:         app=orbit-runner
                orbit/node-type=fargate
                username=amirbo
Annotations:    <none>
Parallelism:    1
Completions:    1
Start Time:     Tue, 09 Mar 2021 21:50:48 +0000
Pods Statuses:  1 Running / 0 Succeeded / 0 Failed
Pod Template:
  Labels:           app=orbit-runner
                    controller-uid=b372bbb2-6698-4d75-9cd9-32434d0b1cb9
                    job-name=orbit-lake-user-fargate-runner-snd75
                    orbit/node-type=fargate
                    username=amirbo
  Service Account:  lake-user
  Containers:
   notebook:
    Image:      495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-dev-env-lake-user-jupyter-user:latest
    Port:       22/TCP
    Host Port:  0/TCP
    Args:
      bash
      -c
      /home/jovyan/.orbit/bootstrap.sh && python /opt/python-utils/notebook_cli.py
 

In [13]:
job_name

'orbit-lake-user-fargate-runner-snd75'

In [14]:
pods = controller.list_current_pods(f'job-name={job_name}')
assert(len(pods)==1)
pod_id = pods[0]['metadata']['name']
print(pod_id)
pods

orbit-lake-user-fargate-runner-snd75-7kbpx


[{'metadata': {'name': 'orbit-lake-user-fargate-runner-snd75-7kbpx',
   'generateName': 'orbit-lake-user-fargate-runner-snd75-',
   'namespace': 'lake-user',
   'selfLink': '/api/v1/namespaces/lake-user/pods/orbit-lake-user-fargate-runner-snd75-7kbpx',
   'uid': 'a092562d-17e7-43e3-ad34-982a8db82835',
   'resourceVersion': '1319171',
   'creationTimestamp': '2021-03-09T21:50:48Z',
   'labels': {'app': 'orbit-runner',
    'controller-uid': 'b372bbb2-6698-4d75-9cd9-32434d0b1cb9',
    'job-name': 'orbit-lake-user-fargate-runner-snd75',
    'orbit/node-type': 'fargate',
    'username': 'amirbo'},
   'annotations': {'kubernetes.io/psp': 'eks.privileged'},
   'ownerReferences': [{'apiVersion': 'batch/v1',
     'kind': 'Job',
     'name': 'orbit-lake-user-fargate-runner-snd75',
     'uid': 'b372bbb2-6698-4d75-9cd9-32434d0b1cb9',
     'controller': True,
     'blockOwnerDeletion': True}],
   'managedFields': [{'manager': 'kube-controller-manager',
     'operation': 'Update',
     'apiVersion':

In [15]:
!kubectl wait --for=condition=Ready --timeout=120s pod/$pod_id

pod/orbit-lake-user-fargate-runner-snd75-7kbpx condition met


In [16]:
controller.delete_job(job_name)

In [17]:
!kubectl wait --for=delete --timeout=120s pod/$pod_id

pod/orbit-lake-user-fargate-runner-snd75-7kbpx condition met


In [18]:
pod_id,job_name

('orbit-lake-user-fargate-runner-snd75-7kbpx',
 'orbit-lake-user-fargate-runner-snd75')

In [19]:
pods = controller.list_current_pods(f'job-name={job_name}')
pods

[]

In [20]:
assert(len(pods)==0)

In [24]:
%%run_notebook
{    
      "tasks":  [
            {
                  "notebookName": "Example-1-SQL-Analysis-Athena.ipynb",
                  "sourcePath": "shared/samples/notebooks/B-DataAnalyst",
                  "targetPath": "shared/regression/notebooks/B-DataAnalyst",
                  "targetPrefix": "ttt",
                  "params": {
                        "glue_db" : "cms_raw_db",
                        "target_db" : "users"
                  }      
            }
      ]  
}

INFO:root:using default profile {'display_name': 'Micro', 'slug': 'micro', 'description': '2 CPU + 2G MEM', 'kubespawner_override': {'cpu_guarantee': 2, 'cpu_limit': 2, 'mem_guarantee': '2G', 'mem_limit': '2G'}, 'default': True}
INFO:root:volumes:[{"name": "efs-volume", "persistentVolumeClaim": {"claimName": "jupyterhub"}}]
INFO:root:volume_mounts:[{"mountPath": "/efs", "name": "efs-volume"}]


{'ExecutionType': 'eks',
 'Identifier': 'orbit-lake-user-fargate-runner-7bql5',
 'NodeType': 'fargate',
 'tasks': [{'notebookName': 'Example-1-SQL-Analysis-Athena.ipynb',
   'sourcePath': 'shared/samples/notebooks/B-DataAnalyst',
   'targetPath': 'shared/regression/notebooks/B-DataAnalyst',
   'targetPrefix': 'ttt',
   'params': {'glue_db': 'cms_raw_db', 'target_db': 'users'}}]}

In [25]:
!kubectl get jobs

NAME                                   COMPLETIONS   DURATION   AGE
orbit-lake-user-fargate-runner-7bql5   0/1           2s         2s


In [26]:
controller.delete_all_my_jobs()

In [27]:
output = !kubectl get jobs
print(output)
assert('No resources found in lake-user namespace.' == output[0])

['No resources found in lake-user namespace.']
